In [ ]:
import requests # for making standard html requests
from bs4 import BeautifulSoup # magical tool for parsing html data
import json # for parsing data
from pandas import DataFrame as df # premier library for data organization
import urllib.request


In [ ]:
page = requests.get("https://kolesa.kz/cars/")
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
cars = []
page = requests.get("https://kolesa.kz/cars/")
page.encoding = 'ISO-885901'
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
print(soup.prettify())

<!DOCTYPE doctype html>
<html lang="en" xmlns:xlink="http://www.w3.org/1999/xlink">
 <head>
  <meta charset="utf-8"/>
  <!--SEO-->
  <title>
   Купить автомобиль в Казахстане. Продажа машин в Казахстане — Колёса
  </title>
  <meta content="Колёса - бесплатные объявления о продаже и покупке БУ автомобилей в Казахстане. Лучшие предложения с ценами в Казахстане. Авторынок подержанных автомобилей." name="description">
   <meta content="купить, автомобиль, казахстане, продажа, машин, колёса" name="keywords"/>
   <meta content="487915631223232" property="fb:pages"/>
   <meta content="3a6b5a2f06b43bce" name="yandex-verification"/>
   <link href="//m.kolesa.kz/cars/" media="only screen and (max-width: 640px)" rel="alternate"/>
   <link href="https://kolesa.kz/cars/?page=2" rel="next">
    <link href="https://kolesa.kz/cars/" rel="canonical"/>
    <meta content="69476cf770e74b47e29b023e1eedfdb9" name="wmail-verification"/>
    <!--SEO-->
    <meta content="Kolesa" name="author"/>
    <link href

In [ ]:
divs = soup.findAll('div', {'class': 'a-info-side col-right-list'})
print(divs)

[<div class="a-info-side col-right-list"><div class="a-info-top"><span class="a-el-info-title"><a class="list-link ddl_product_link" data-list-id="main" data-product-id="107761843" href="/a/show/107761843" target="_blank">    Mercedes-Benz E 240</a></span><span class="favorites"><div class="tooltip-container"><i class="icon-fav-act icon-fav-off" data-id="107761843"></i><div class="tooltip">
            Добавить в избранное        </div></div><div class="tooltip-container"><i class="icon-note-act icon-note" data-id="107761843"></i><div class="tooltip">
            Оставить заметку        </div><div class="advert-note-editor hide" id="advert-note-editor-107761843"><div class="header"><span class="icon-note-act icon-note" data-id="107761843"></span><a href="/a/my/">
                    все заметки                </a></div><div class="content"><form action="#" method="post"><textarea data-id="107761843" id="noteBody_107761843" name="note" placeholder="Оставить заметку."></textarea></form><

In [ ]:
attr_to_index = {'Марка':0, 
                 'Год':1, 
                 'Цена':2, 
                 'Город':3, 
                 'Кузов':4,
                 'Объем двигателя, л':5,
                 'Пробег':6,
                 'Коробка передач':7,
                 'Руль':8,
                 'Цвет':9,
                 'Привод':10,
                 'Растаможен':11,
                 'VIN':12,
                 'Бензин':13}
           




In [ ]:
    divs = soup.findAll('div', {'class': 'a-info-side col-right-list'})
    for div in divs:
        
        line = [None for _ in range(14)]
        line[0] = div.find('a', {'class': 'list-link ddl_product_link'}).string
#         line.append(div.find('div', {'class': 'list-region'}).string.strip())
        line[1] = div.find('div', {'class': 'a-search-description'}).string.strip()[:4]
        line[2] = str(div.find('span', {'class': 'price'}).text).strip()[:-2].replace(u'\xa0', u'')
        href = div.find('a', href=True)["href"]
        car_url = "https://kolesa.kz{}".format(href)
        car_source = urllib.request.urlopen(car_url).read()
        car_soup = BeautifulSoup(car_source)
        car_attr = car_soup.findAll('div', {'class': 'offer__parameters'})[0]
        car_attr = car_attr.findAll('dl')
#         print(car_attr)
        for attr in car_attr:
            attr_type = attr.find('dt')
#             print(attr_type)
            if attr_type.string == "Наличие":
                continue
            try:
                attr_type = attr_type['title']
            except:
                continue
            
            if (attr_type == "Объем двигателя, л"):
                engine_volume = attr.find('dd').string.split()[0]
                is_petroleum = attr.find('dd').string.split()[1]
                line[attr_to_index[attr_type]] = engine_volume
                line[attr_to_index["Бензин"]] = is_petroleum[1:-1]
            elif (attr_type == "Пробег"):
                attr_str = attr.find('dd').string.strip()[:-3]
                val_str = "".join(attr_str.split())
                line[attr_to_index[attr_type]] = int(val_str)
            else:
                attr_value = attr.find('dd').string.strip()
                try:
                    line[attr_to_index[attr_type]] = attr_value
                except:
                    continue
            
        print(line)
        cars.append(line)

['    Mercedes-Benz E 240', '2003', '3800000', 'Павлодар', 'седан', '2.4', 207200, 'автомат', 'слева', 'синий металлик', 'задний привод', None, None, 'бензин']
['    УАЗ 3303', '2014', '3500000', 'Уральск', 'фургон', '2.7', None, 'механика', 'слева', 'серый', 'полный привод', None, None, 'газ-бензин']
['    Mitsubishi Outlander', '2012', '6800000', 'Караганда', 'кроссовер', '2', 164000, 'вариатор', 'слева', 'белый металлик', 'полный привод', None, None, 'бензин']
['    Toyota Estima', '2008', '6000000', 'Нур-Султан (Астана)', 'минивэн', '2.4', 97211, 'вариатор', 'справа', 'белый', 'передний привод', None, None, 'бензин']
['    Mercedes-Benz E 320', '2004', '3500000', 'Алматы', 'универсал', '3.2', 150000, 'автомат', 'слева', 'синий металлик', None, None, None, 'бензин']
['    Audi A6', '1999', '2400000', 'Кокшетау', 'седан', '2.8', None, 'механика', 'слева', 'синий', 'передний привод', None, None, 'бензин']
['    ВАЗ (Lada) 2107', '2007', '450000', 'Жанаозен', 'седан', '1.5', None, 'мех

In [ ]:
import csv
with open("kolesa_cars.csv", 'w', newline='\n') as myfile:
    wr = csv.writer(myfile)
    for c in cars:
        wr.writerow(c)

In [ ]:
print(cars[0:2])

[['    Mercedes-Benz E 240', '2003', '3800000', 'Павлодар', 'седан', '2.4', 207200, 'автомат', 'слева', 'синий металлик', 'задний привод', None, None, 'бензин'], ['    УАЗ 3303', '2014', '3500000', 'Уральск', 'фургон', '2.7', None, 'механика', 'слева', 'серый', 'полный привод', None, None, 'газ-бензин']]


In [ ]:
cars_with_probeg = []
for c in cars:
    if (c[6] != None):
        cars_with_probeg.append(c)

In [ ]:
len(cars_with_probeg)

14

In [ ]:
import copy

cars_copy = copy.copy(cars_with_probeg)

In [ ]:
cars = cars_with_probeg

import pandas as pd
import numpy as np
import math

In [ ]:
attr_to_index = {'Марка':0, 
                 'Год':1, 
                 'Цена':2, 
                 'Город':3, 
                 'Кузов':4,
                 'Объем двигателя, л':5,
                 'Пробег':6,
                 'Коробка передач':7,
                 'Руль':8,
                 'Цвет':9,
                 'Привод':10,
                 'Растаможен':11,
                 'VIN':12,
                 'Бензин':13}

In [ ]:
X = None
for idx in [0, 3, 4, 7, 8, 9, 11, 13]:    
    categorical_feats = [c[idx] for c in cars]
#     for f in categorical_feats:
#         if (f == None):
#             print("omitting feature {}".format(idx))
    unique_feats = set(categorical_feats)
    print(len(unique_feats))
#     if idx in [7,8,9,10,11,13]:
    print(unique_feats)
    s = pd.Series(categorical_feats)
    dummies = pd.get_dummies(s)
    if X is None:
        X = dummies.values
    else:
        X = np.append(X, dummies.values, axis=1)
    print("X shape = {}".format(X.shape))
    print("dummies shape = {}".format(dummies.values.shape))

13
{'    Toyota Camry', '    BMW 325', '    Volkswagen Crafter', '    Mercedes-Benz E 240', '    Nissan Primera', '    Mercedes-Benz GLK 300', '    Ravon R4', '    Mercedes-Benz E 320', '    Mitsubishi Outlander', '    Toyota Estima', '    Mercedes-Benz C 180', '    Mercedes-Benz E 350', '    ВАЗ (Lada) 2114 (хэтчбек)'}
X shape = (14, 13)
dummies shape = (14, 13)
8
{'Актау', 'Павлодар', 'Алматы', 'Петропавловск', 'Шымкент', 'Кокшетау', 'Караганда', 'Нур-Султан (Астана)'}
X shape = (14, 21)
dummies shape = (14, 8)
6
{'хэтчбек', 'минивэн', 'фургон', 'седан', 'кроссовер', 'универсал'}
X shape = (14, 27)
dummies shape = (14, 6)
4
{'типтроник', 'автомат', 'вариатор', 'механика'}
X shape = (14, 31)
dummies shape = (14, 4)
2
{'справа', 'слева'}
X shape = (14, 33)
dummies shape = (14, 2)
7
{'серый', 'белый', 'черный металлик', 'серебристый', 'белый металлик', 'серебристый металлик', 'синий металлик'}
X shape = (14, 40)
dummies shape = (14, 7)
1
{None}
X shape = (14, 40)
dummies shape = (14, 0)

In [ ]:
for idx in [1, 5, 6]:
    numeric_feats = [c[idx] for c in cars]
    numeric_feats = np.expand_dims(np.array(numeric_feats), axis=1)
    print("X shape = {}".format(X.shape))
    print("features shape = {}".format(numeric_feats.shape))
    X = np.append(X, numeric_feats, axis=1)

y = np.array([c[2] for c in cars])

X shape = (14, 43)
features shape = (14, 1)
X shape = (14, 44)
features shape = (14, 1)
X shape = (14, 45)
features shape = (14, 1)


In [ ]:
dataset_size = X.shape[0]
train_size = math.floor(dataset_size * 0.8)
test_size = dataset_size - train_size
print("train_size = {}".format(train_size))
print("test_size = {}".format(test_size))
print("dataset_size = {}".format(dataset_size))

X_train = X[:train_size].astype(np.float32)
y_train = y[:train_size].astype(np.float32)

X_test = X[train_size:].astype(np.float32)
y_test = y[train_size:].astype(np.float32)

print("X_train = {}".format(X_train.shape))

train_size = 24
test_size = 6
dataset_size = 30
X_train = (24, 49)
